# Ducklake experiments

In this notebook we'll try to create and use a Ducklake catalog:

- Minio will be used as data storage
- Postgres will be used as metadata storage

In [ ]:
import duckdb
import ibis

from poor_man_lakehouse.config import settings

ibis.options.interactive = True

con = duckdb.connect()
con.install_extension("ducklake")
con.install_extension("postgres")
con.sql("UPDATE EXTENSIONS;")
con.load_extension("ducklake")
con.load_extension("postgres")

con.sql(f"""
CREATE OR REPLACE SECRET s3_secret (
        TYPE S3,
        KEY_ID '{settings.AWS_ACCESS_KEY_ID}',
        SECRET '{settings.AWS_SECRET_ACCESS_KEY}',
        ENDPOINT '{settings.AWS_ENDPOINT_URL}',
        URL_STYLE 'path',
        USE_SSL false
        );
""")

con.sql(f"""
CREATE OR REPLACE SECRET postgres_secret (
        TYPE postgres,
        HOST '{settings.POSTGRES_HOST}',
        DATABASE '{settings.POSTGRES_DB}',
        USER '{settings.POSTGRES_USER}',
        PASSWORD '{settings.POSTGRES_PASSWORD}'
        );
""")

con.sql(f"""
CREATE OR REPLACE SECRET ducklake_secret (
        TYPE ducklake,
        METADATA_PATH '',
        DATA_PATH '{settings.WAREHOUSE_BUCKET}ducklake/',
        METADATA_PARAMETERS MAP {{'TYPE': 'postgres', 'SECRET': 'postgres_secret'}}
        );
""")
con.sql("""
ATTACH OR REPLACE 'ducklake:ducklake_secret' AS my_ducklake;
""")

In [2]:
con.sql("""CREATE OR REPLACE TABLE nl_train_stations AS
    FROM 'https://blobs.duckdb.org/nl_stations.csv';""")

In [3]:
con.sql("""SELECT * from nl_train_stations""")

┌───────┬─────────┬─────────┬────────────┬──────────────────┬───────────────────────────┬───────────────────────────┬─────────┬───────────────────────────┬─────────────────┬─────────────────┐
│  id   │  code   │   uic   │ name_short │   name_medium    │         name_long         │           slug            │ country │           type            │     geo_lat     │     geo_lng     │
│ int64 │ varchar │  int64  │  varchar   │     varchar      │          varchar          │          varchar          │ varchar │          varchar          │     double      │     double      │
├───────┼─────────┼─────────┼────────────┼──────────────────┼───────────────────────────┼───────────────────────────┼─────────┼───────────────────────────┼─────────────────┼─────────────────┤
│   266 │ HT      │ 8400319 │ Den Bosch  │ 's-Hertogenbosch │ 's-Hertogenbosch          │ s-hertogenbosch           │ NL      │ knooppuntIntercitystation │        51.69048 │         5.29362 │
│   269 │ HTO     │ 8400320 │ Dn Bosch O

In [ ]:
import ibis

from poor_man_lakehouse.config import settings

ibis.options.interactive = True

con = ibis.duckdb.connect(extensions="ducklake,postgres")


con.raw_sql(f"""
CREATE OR REPLACE SECRET s3_secret (
        TYPE S3,
        KEY_ID '{settings.AWS_ACCESS_KEY_ID}',
        SECRET '{settings.AWS_SECRET_ACCESS_KEY}',
        ENDPOINT '{settings.AWS_ENDPOINT_URL.replace("http://", "")}',
        URL_STYLE 'path',
        USE_SSL false
        );
""")

con.raw_sql(f"""
CREATE OR REPLACE SECRET postgres_secret (
        TYPE postgres,
        HOST '{settings.POSTGRES_HOST}',
        DATABASE '{settings.POSTGRES_DB}',
        USER '{settings.POSTGRES_USER}',
        PASSWORD '{settings.POSTGRES_PASSWORD}'
        );
""")

con.raw_sql(f"""
CREATE OR REPLACE SECRET ducklake_secret (
        TYPE ducklake,
        METADATA_PATH '',
        DATA_PATH '{settings.WAREHOUSE_BUCKET}ducklake/',
        METADATA_PARAMETERS MAP {{'TYPE': 'postgres', 'SECRET': 'postgres_secret'}}
        );
""")
con.raw_sql("ATTACH OR REPLACE 'ducklake:ducklake_secret' as my_ducklake;")
con.raw_sql("USE my_ducklake")
con.list_catalogs()

['__ducklake_metadata_my_ducklake', 'memory', 'my_ducklake', 'system', 'temp']

In [6]:
con.raw_sql("""CREATE OR REPLACE TABLE nl_train_stations AS
    FROM 'https://blobs.duckdb.org/nl_stations.csv';""")


In [7]:
con.table("nl_train_stations")

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ id    ┃ code   ┃ uic     ┃ name_short ┃ name_medium      ┃ name_long                 ┃ slug                      ┃ country ┃ type                      ┃ geo_lat   ┃ geo_lng  ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64 │ string │ int64   │ string     │ string           │ string                    │ string                    │ string  │ string                    │ float64   │ float64  │
├───────┼────────┼─────────┼────────────┼──────────────────┼───────────────────────────┼───────────────────────────┼─────────┼───────────────────────────┼───────────┼──────────┤
│   266 │ HT     │ 8400319 │ Den Bosch  │ 's-Hertogenbosch │ 's-Hertogenbosch          │ s-hertogenbosch           │ NL      │ knooppuntIntercitystation │ 51.690480 │ 5.293620 │
│   269 │ HTO    │ 8400320 │ Dn Bosch O │ 's-Hertogenb. O. │ 's-Hertogenbosch Oost     │ s-hertogenbosch-oost      │ NL      │ stoptreinstation          │ 51.700554 │ 5.318333 │
│   227 │ HDE    │ 8400388 │ 't Harde   │ 't Harde         │ 't Harde                  │ t-harde                   │ NL      │ stoptreinstation          │ 52.409168 │ 5.893611 │
│     8 │ AHBF   │ 8015345 │ Aachen     │ Aachen Hbf       │ Aachen Hbf                │ aachen-hbf                │ D       │ knooppuntIntercitystation │ 50.767800 │ 6.091499 │
│   818 │ AW     │ 8015199 │ Aachen W   │ Aachen West      │ Aachen West               │ aachen-west               │ D       │ stoptreinstation          │ 50.780360 │ 6.070715 │
│    51 │ ATN    │ 8400045 │ Aalten     │ Aalten           │ Aalten                    │ aalten                    │ NL      │ stoptreinstation          │ 51.921327 │ 6.578627 │
│     5 │ AC     │ 8400047 │ Abcoude    │ Abcoude          │ Abcoude                   │ abcoude                   │ NL      │ stoptreinstation          │ 52.278500 │ 4.977000 │
│   550 │ EAHS   │ 8021123 │ Ahaus      │ Ahaus            │ Ahaus                     │ ahaus                     │ D       │ stoptreinstation          │ 52.079796 │ 7.016358 │
│    12 │ AIME   │ 8774176 │ Aime-la-Pl │ Aime-la-Plagne   │ Aime-la-Plagne            │ aime-la-plagne            │ F       │ intercitystation          │ 45.554380 │ 6.648690 │
│   819 │ ACDG   │ 8727149 │ Airport dG │ Airport deGaulle │ Airport Charles de Gaulle │ airport-charles-de-gaulle │ F       │ knooppuntIntercitystation │ 49.004048 │ 2.571133 │
│     … │ …      │       … │ …          │ …                │ …                         │ …                         │ …       │ …                         │         … │        … │
└───────┴────────┴─────────┴────────────┴──────────────────┴───────────────────────────┴───────────────────────────┴─────────┴───────────────────────────┴───────────┴──────────┘